In [4]:
import numpy as np
import pandas as pd
import json
import datetime
import spacy
from tqdm import tqdm
from spacy.language import Language
from spacy_langdetect import LanguageDetector
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import plotly.express as px

In [5]:
f = open('./Telegram_Data/result.json', encoding='utf8')
data = json.load(f)
for key in data.keys():
    print(key)

name
type
id
messages


In [42]:
df = pd.DataFrame(data['messages'])
df = df.iloc[0:10000]
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 26 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   10000 non-null  int64  
 1   type                 10000 non-null  object 
 2   date                 10000 non-null  object 
 3   from                 9060 non-null   object 
 4   from_id              9986 non-null   object 
 5   reply_to_message_id  5262 non-null   float64
 6   text                 10000 non-null  object 
 7   edited               217 non-null    object 
 8   file                 11 non-null     object 
 9   thumbnail            11 non-null     object 
 10  media_type           11 non-null     object 
 11  sticker_emoji        5 non-null      object 
 12  width                35 non-null     float64
 13  height               35 non-null     float64
 14  forwarded_from       40 non-null     object 
 15  actor                12 non-null     

,id,type,date,from,from_id,reply_to_message_id,text,edited,file,thumbnail,...,actor_id,action,members,photo,message_id,contact_information,contact_vcard,mime_type,duration_seconds,via_bot
0,1903163,message,2021-05-01T00:00:14,Seflipe,user1697045867,1903101.0,Double check the address,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1903164,message,2021-05-01T00:01:33,Seflipe,user1697045867,1903138.0,To get help to understand the product not how ...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1903168,message,2021-05-01T00:03:13,Seflipe,user1697045867,1903122.0,You can find good youtube channels.. Check the...,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1903188,message,2021-05-01T00:13:37,Neto | Crypto.com,user1395411322,1903160.0,"Are you doing withdrawals to another user, DeF...",NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1903191,message,2021-05-01T00:14:10,PauLaLa,user1501886330,NaN,Yes,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:
def get_lang_detector(nlp, name):
    return LanguageDetector()

nlp = spacy.load("en_core_web_sm")
Language.factory("language_detector", func=get_lang_detector)
nlp.add_pipe('language_detector', last=True)

In [44]:
def detect_lang(x):
    doc = nlp(x)
    lang_dict = doc._.language
    language = lang_dict['language']
    return language
df['language'] = [detect_lang(str(x)) for x in tqdm(df['text'])]

100%|██████████| 10000/10000 [05:09<00:00, 32.31it/s]


In [45]:
df[['text', 'language']].head()

,text,language
0,Double check the address,en
1,To get help to understand the product not how ...,en
2,You can find good youtube channels.. Check the...,en
3,"Are you doing withdrawals to another user, DeF...",en
4,Yes,tr


In [46]:
df.language.unique()

array(['en', 'tr', 'it', 'es', 'UNKNOWN', 'nl', 'fi', 'ro', 'sv', 'no',
       'hu', 'ca', 'tl', 'da', 'fr', 'cy', 'so', 'vi', 'af', 'sl', 'sk',
       'sw', 'pl', 'id', 'pt', 'de', 'hr', 'et', 'sq', 'cs', 'ar', 'lt',
       'lv'], dtype=object)

In [50]:
df_eng = df[df.language.values == 'en']
df_new = df_eng.filter(items = ['id','text', 'date'])
df_new = df_new[df_new['text'].str.lower().str.contains('bitcoin|doge|btc') & (df_new['text'].str.len() > 1)]

In [51]:
df_new

,id,text,date
36,1903358,Bitcoin,2021-05-01T01:04:38
118,1903552,"Thank you! I was just wondering, after factori...",2021-05-01T01:54:32
119,1903555,"The 0.0004 BTC withdrawal fee is flat, and the...",2021-05-01T01:55:37
123,1903565,For example let’s say I have $1000sgd worth of...,2021-05-01T02:02:14
125,1903568,Is it the same amount of BTC still?\n\nMinus t...,2021-05-01T02:03:15
...,...,...,...
9890,1933160,"Wont be until larger bear, or big money flows ...",2021-05-05T05:18:55
9893,1933167,"If you want to buy to make money, i think it'...",2021-05-05T05:20:49
9897,1933174,"Just look at doge/btc chart week/monthly, and ...",2021-05-05T05:21:50
9952,1933337,should i invest in bitcoin or dogecoin,2021-05-05T05:41:38
